In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import sys
from nba_api.stats.endpoints import PlayerGameLog, TeamGameLog, GameRotation, BoxScoreTraditionalV2
from nba_api.stats.static import players, teams
from nba_api.live.nba.endpoints import playbyplay
import time

print('packages loaded')

packages loaded


In [2]:
filepath = '../data/2021_22_pbp_data.csv'
pbp = pd.read_csv(filepath, index_col=0, dtype={'game_id': str})
pbp.sample(5)

,actionNumber,clock,timeActual,period,periodType,actionType,subType,qualifiers,personId,x,...,reboundOffensiveTotal,officialId,foulPersonalTotal,foulTechnicalTotal,foulDrawnPlayerName,foulDrawnPersonId,blockPlayerName,blockPersonId,value,game_id
479,583,PT06M31.00S,2021-12-09T02:45:03.7Z,4,REGULAR,timeout,full,"['team', 'mandatory']",0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0022100370
101,119,PT03M51.00S,2022-01-29T00:58:32.7Z,1,REGULAR,rebound,defensive,[],1628381,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0022100737
30,40,PT09M30.00S,2021-12-14T00:15:27.9Z,1,REGULAR,rebound,defensive,[],200782,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0022100404
255,306,PT03M14.00S,2021-11-17T01:27:13.2Z,2,REGULAR,2pt,Layup,['pointsinthepaint'],203952,5.206965,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0022100210
384,473,PT00M55.00S,2022-03-01T01:55:44.9Z,3,REGULAR,foul,personal,[],1626224,NaN,...,NaN,204058.0,3.0,0.0,Reid,1629675.0,NaN,NaN,NaN,0022100920


In [3]:
curr_season = '2021-22'

In [4]:
pbp.columns

Index(['actionNumber', 'clock', 'timeActual', 'period', 'periodType',
       'actionType', 'subType', 'qualifiers', 'personId', 'x', 'y',
       'possession', 'scoreHome', 'scoreAway', 'edited', 'orderNumber',
       'xLegacy', 'yLegacy', 'isFieldGoal', 'side', 'description',
       'personIdsFilter', 'teamId', 'teamTricode', 'descriptor',
       'jumpBallRecoveredName', 'jumpBallRecoverdPersonId', 'playerName',
       'playerNameI', 'jumpBallWonPlayerName', 'jumpBallWonPersonId',
       'jumpBallLostPlayerName', 'jumpBallLostPersonId', 'shotDistance',
       'shotResult', 'pointsTotal', 'assistPlayerNameInitial',
       'assistPersonId', 'assistTotal', 'turnoverTotal', 'stealPlayerName',
       'stealPersonId', 'shotActionNumber', 'reboundTotal',
       'reboundDefensiveTotal', 'reboundOffensiveTotal', 'officialId',
       'foulPersonalTotal', 'foulTechnicalTotal', 'foulDrawnPlayerName',
       'foulDrawnPersonId', 'blockPlayerName', 'blockPersonId', 'value',
       'game_id'],
      

In [5]:
pbp[pbp['subType'].notnull()]['descriptor'].sample(5)

298    NaN
434    NaN
516    NaN
214    NaN
101    NaN
Name: descriptor, dtype: object

In [6]:
pbp['subType'].sample(5)

321        defensive
492              out
464           1 of 1
41     out-of-bounds
423            Layup
Name: subType, dtype: object

In [7]:
block_cols = [
    'clock', 'timeActual', 'period', 'actionType', 'subType', 'x', 'y', 'possession',
    'scoreHome', 'scoreAway', 'xLegacy', 'yLegacy', 'isFieldGoal', 'shotResult', 'side',
    'teamTricode', 'playerNameI', 'playerName', 'shotDistance', 'blockPlayerName'
]

In [7]:
pbp['scoreMargin'] = pbp['scoreHome'] - pbp['scoreAway']
pbp['abScoreMargin'] = np.abs(pbp['scoreMargin'])

In [8]:
pbp.dtypes

actionNumber                  int64
clock                        object
timeActual                   object
period                        int64
periodType                   object
actionType                   object
subType                      object
qualifiers                   object
personId                      int64
x                           float64
y                           float64
possession                    int64
scoreHome                     int64
scoreAway                     int64
edited                       object
orderNumber                   int64
xLegacy                     float64
yLegacy                     float64
isFieldGoal                   int64
side                         object
description                  object
personIdsFilter              object
teamId                      float64
teamTricode                  object
descriptor                   object
jumpBallRecoveredName        object
jumpBallRecoverdPersonId    float64
playerName                  

In [9]:
pbp.sample(5)

,actionNumber,clock,timeActual,period,periodType,actionType,subType,qualifiers,personId,x,...,reboundOffensiveTotal,officialId,foulPersonalTotal,foulTechnicalTotal,foulDrawnPlayerName,foulDrawnPersonId,blockPlayerName,blockPersonId,value,game_id
382,452,PT02M38.00S,2021-10-22T01:05:57.4Z,3,REGULAR,substitution,out,[],1628381,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0022100014
175,214,PT08M20.00S,2021-10-27T01:19:22.8Z,2,REGULAR,rebound,offensive,[],1628973,NaN,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0022100052
442,529,PT06M35.00S,2021-11-19T02:31:40.8Z,4,REGULAR,substitution,out,[],203490,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0022100224
570,734,PT01M49.00S,2022-02-11T02:23:12.7Z,4,REGULAR,block,NaN,[],1628977,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0022100831
145,173,PT12M00.00S,2022-02-04T03:39:21.2Z,2,REGULAR,substitution,out,['startperiod'],203084,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0022100785


In [10]:
gobert_blocks = pbp[pbp['blockPlayerName'] == 'Gobert']
gobert_blocks.sample(5)

,actionNumber,clock,timeActual,period,periodType,actionType,subType,qualifiers,personId,x,...,reboundOffensiveTotal,officialId,foulPersonalTotal,foulTechnicalTotal,foulDrawnPlayerName,foulDrawnPersonId,blockPlayerName,blockPersonId,value,game_id
501,605,PT02M45.00S,2022-02-27T22:49:49.2Z,4,REGULAR,2pt,Layup,['pointsinthepaint'],1629028,93.249014,...,NaN,NaN,NaN,NaN,NaN,NaN,Gobert,203497.0,NaN,0022100913
303,347,PT10M32.00S,2021-11-25T02:25:15.3Z,3,REGULAR,2pt,Layup,['pointsinthepaint'],1630581,5.732589,...,NaN,NaN,NaN,NaN,NaN,NaN,Gobert,203497.0,NaN,0022100276
22,29,PT08M53.00S,2022-01-18T03:46:11.2Z,1,REGULAR,2pt,Layup,['pointsinthepaint'],2772,8.327858,...,NaN,NaN,NaN,NaN,NaN,NaN,Gobert,203497.0,NaN,0022100665
135,162,PT10M48.00S,2021-12-09T01:39:03.0Z,2,REGULAR,2pt,Layup,['pointsinthepaint'],1627736,92.066360,...,NaN,NaN,NaN,NaN,NaN,NaN,Gobert,203497.0,NaN,0022100374
268,311,PT00M44.30S,2022-03-19T02:04:13.0Z,2,REGULAR,2pt,Layup,['pointsinthepaint'],1628400,92.986202,...,NaN,NaN,NaN,NaN,NaN,NaN,Gobert,203497.0,NaN,0022101055


In [11]:
# shot distance when player is in and out of the game
# use all minutes
# keep replacements in mind when writing

In [12]:
from nba_api.stats.static import players

players.find_players_by_full_name('rudy gobert')

[{'id': 203497,
  'full_name': 'Rudy Gobert',
  'first_name': 'Rudy',
  'last_name': 'Gobert',
  'is_active': True}]

# Steps

1. find player by name
2. find the team(s) this player played for
3. find the games played in by specified player
4. find team games played without specified player
5. get rotation data for player
6. look at all minutes with player - specifically, shot distance
7. look at all minutes without player - specifically shot distance, maybe fouls

In [13]:
# player info
name = 'rudy gobert'
player_id = players.find_players_by_full_name(name)[0]['id']

In [14]:
# teams player has played for this season
gamelogs = PlayerGameLog(player_id=player_id, season=curr_season).get_data_frames()[0]

# get all games played in
player_game_ids = list(gamelogs['Game_ID'])

In [15]:
sample_boxscore = BoxScoreTraditionalV2(
    game_id=player_game_ids[5]
).get_data_frames()[0]
time.sleep(3)
# get team ID
team_id = sample_boxscore[sample_boxscore['PLAYER_ID'] == player_id]['TEAM_ID'].iloc[0]

In [16]:
team_gamelogs = TeamGameLog(
    team_id=team_id,
    season=curr_season
).get_data_frames()[0]

In [17]:
team_gamelogs['HOME'] = team_gamelogs['MATCHUP'].apply(
    lambda matchup: 1 if '@' not in matchup else 0
)

team_gamelogs

,Team_ID,Game_ID,GAME_DATE,MATCHUP,WL,W,L,W_PCT,MIN,FGM,...,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS,HOME
0,1610612762,0022101067,"MAR 20, 2022",UTA @ NYK,None,NaN,NaN,NaN,0,3,...,2,5,7,2,0,1,1,4,11,0
1,1610612762,0022101055,"MAR 18, 2022",UTA vs. LAC,W,44.0,26.0,0.629,240,47,...,15,44,59,30,9,6,12,18,121,1
2,1610612762,0022101041,"MAR 16, 2022",UTA vs. CHI,W,43.0,26.0,0.623,240,42,...,10,37,47,20,8,6,16,18,125,1
3,1610612762,0022101027,"MAR 14, 2022",UTA vs. MIL,L,42.0,26.0,0.618,240,37,...,13,32,45,21,7,4,13,23,111,1
4,1610612762,0022101010,"MAR 12, 2022",UTA vs. SAC,W,42.0,25.0,0.627,240,48,...,10,34,44,21,4,5,9,20,134,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,1610612762,0022100083,"OCT 30, 2021",UTA @ CHI,L,4.0,1.0,0.800,240,35,...,14,40,54,15,5,7,20,21,99,0
67,1610612762,0022100068,"OCT 28, 2021",UTA @ HOU,W,4.0,0.0,1.000,240,41,...,10,48,58,24,8,4,16,23,122,0
68,1610612762,0022100054,"OCT 26, 2021",UTA vs. DEN,W,3.0,0.0,1.000,240,44,...,8,33,41,20,5,6,13,22,122,1
69,1610612762,0022100026,"OCT 22, 2021",UTA @ SAC,W,2.0,0.0,1.000,240,37,...,13,38,51,16,14,3,17,17,110,0


In [18]:
games_not_played = [game_id for game_id in list(team_gamelogs['Game_ID'])
                    if game_id not in player_game_ids]

In [19]:
len(team_gamelogs['Game_ID'])

71

In [20]:
# check if game numbers match up
len(player_game_ids) + len(games_not_played)


71

In [21]:
# 0 for away team
# 1 for home team
test_game_id = game_id=player_game_ids[5]
home_or_away = team_gamelogs[team_gamelogs['Game_ID'] == test_game_id]['HOME'].iloc[0]
if home_or_away == 1:
    df_idx = 1
else:
    df_idx = 0

rotation = GameRotation(
    game_id=player_game_ids[5]
).get_data_frames()[df_idx]


In [29]:
# rotation time is divided by 10 to get into seconds in game
rotation[['IN_TIME_REAL', 'OUT_TIME_REAL']] = rotation[['IN_TIME_REAL', 'OUT_TIME_REAL']].apply(lambda row: row/10, axis=1)
rotation[rotation['PERSON_ID'] == player_id]

,GAME_ID,TEAM_ID,TEAM_CITY,TEAM_NAME,PERSON_ID,PLAYER_FIRST,PLAYER_LAST,IN_TIME_REAL,OUT_TIME_REAL,PLAYER_PTS,PT_DIFF,USG_PCT
10,0022100990,1610612762,Utah,Jazz,203497,Rudy,Gobert,0.0,288.0,0,5.0,0.000
11,0022100990,1610612762,Utah,Jazz,203497,Rudy,Gobert,513.0,947.0,5,0.0,0.286
12,0022100990,1610612762,Utah,Jazz,203497,Rudy,Gobert,1200.0,1812.0,4,19.0,0.143


In [23]:
sample_boxscore

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,MIN,...,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
0,0022100990,1610612757,POR,Portland,1630190,Elijah Hughes,Elijah,F,,21:01,...,1.0,4.0,5.0,2.0,1.0,0.0,1.0,1.0,0.0,-19.0
1,0022100990,1610612757,POR,Portland,1629604,CJ Elleby,CJ,F,,23:48,...,0.0,4.0,4.0,2.0,0.0,0.0,1.0,3.0,10.0,-8.0
2,0022100990,1610612757,POR,Portland,1629234,Drew Eubanks,Drew,C,,18:47,...,3.0,4.0,7.0,0.0,1.0,1.0,0.0,1.0,4.0,-19.0
3,0022100990,1610612757,POR,Portland,1628404,Josh Hart,Josh,G,,26:12,...,3.0,3.0,6.0,3.0,1.0,0.0,1.0,1.0,5.0,-51.0
4,0022100990,1610612757,POR,Portland,1630314,Brandon Williams,Brandon,G,,26:00,...,0.0,6.0,6.0,1.0,0.0,0.0,4.0,3.0,9.0,-26.0
5,0022100990,1610612757,POR,Portland,1629833,Keljin Blevins,Keljin,,,20:50,...,0.0,1.0,1.0,3.0,0.0,0.0,0.0,2.0,7.0,-24.0
6,0022100990,1610612757,POR,Portland,1630570,Trendon Watford,Trendon,,,29:13,...,1.0,2.0,3.0,0.0,0.0,1.0,0.0,3.0,22.0,-19.0
7,0022100990,1610612757,POR,Portland,1630535,Greg Brown III,Greg,,,25:19,...,1.0,2.0,3.0,3.0,0.0,0.0,1.0,4.0,4.0,-4.0
8,0022100990,1610612757,POR,Portland,1630553,Keon Johnson,Keon,,,29:42,...,0.0,1.0,1.0,5.0,1.0,0.0,0.0,2.0,15.0,0.0
9,0022100990,1610612757,POR,Portland,203463,Ben McLemore,Ben,,,19:08,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,9.0,-20.0


In [24]:
test_game_id

'0022100990'

In [40]:
# sample pbp
test_pbp = pbp[pbp['game_id'] == test_game_id].copy()
test_pbp

,actionNumber,clock,timeActual,period,periodType,actionType,subType,qualifiers,personId,x,...,reboundOffensiveTotal,officialId,foulPersonalTotal,foulTechnicalTotal,foulDrawnPlayerName,foulDrawnPersonId,blockPlayerName,blockPersonId,value,game_id
0,2,PT12M00.00S,2022-03-10T03:11:03.6Z,1,REGULAR,period,start,[],0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0022100990
1,4,PT11M58.00S,2022-03-10T03:11:05.4Z,1,REGULAR,jumpball,recovered,[],1626220,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0022100990
2,7,PT11M38.00S,2022-03-10T03:11:24.4Z,1,REGULAR,3pt,Jump Shot,[],1628378,71.698423,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0022100990
3,8,PT11M36.00S,2022-03-10T03:11:26.4Z,1,REGULAR,rebound,defensive,[],1628404,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0022100990
4,9,PT11M25.00S,2022-03-10T03:11:37.5Z,1,REGULAR,2pt,Jump Shot,[],1630314,8.886334,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0022100990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537,651,PT00M19.50S,2022-03-10T05:23:26.8Z,4,REGULAR,stoppage,out-of-bounds,[],0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0022100990
538,652,PT00M16.70S,2022-03-10T05:23:34.9Z,4,REGULAR,3pt,Jump Shot,['2ndchance'],1630553,88.255585,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0022100990
539,653,PT00M14.00S,2022-03-10T05:23:37.6Z,4,REGULAR,rebound,defensive,[],1627823,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0022100990
540,654,PT00M00.00S,2022-03-10T05:23:52.9Z,4,REGULAR,period,end,[],0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0022100990


In [59]:
# try to filter out pbp rows where player is in the game based on game rotation
# clock reads PT00M19.50S
# period is in [1, 2, 3, 4]
# convert pbp time to seconds
test_time = 'PT00M19.50S'
mins = test_time[2:4]
seconds = test_time[5:10]
print(mins, seconds)

# def conv_clock_to_secs(clock_str, period):
#     period = (period-1) * (12*60) # 12 minutes * 60 seconds
#     mins = clock_str[2:4]
#     mins_to_secs = int(mins)*60
#     secs = float(clock_str[5:10])
#     return period+mins_to_secs+secs

def conv_clock_to_secs(row):
    period_to_secs = (int(row['period']) -1) * (12*60)
    mins_to_secs = int(row['clock'][2:4]) * 60
    secs = float(row['clock'][5:10])
    secs_left_in_quarter = (12*60) - (secs+mins_to_secs)
    return period_to_secs + secs_left_in_quarter

00 19.50


In [60]:
conv_clock_to_secs(test_pbp.iloc[1])


2.0

In [61]:
test_pbp['time'] = test_pbp.apply(conv_clock_to_secs, axis=1)
test_pbp

,actionNumber,clock,timeActual,period,periodType,actionType,subType,qualifiers,personId,x,...,officialId,foulPersonalTotal,foulTechnicalTotal,foulDrawnPlayerName,foulDrawnPersonId,blockPlayerName,blockPersonId,value,game_id,time
0,2,PT12M00.00S,2022-03-10T03:11:03.6Z,1,REGULAR,period,start,[],0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0022100990,0.0
1,4,PT11M58.00S,2022-03-10T03:11:05.4Z,1,REGULAR,jumpball,recovered,[],1626220,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0022100990,2.0
2,7,PT11M38.00S,2022-03-10T03:11:24.4Z,1,REGULAR,3pt,Jump Shot,[],1628378,71.698423,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0022100990,22.0
3,8,PT11M36.00S,2022-03-10T03:11:26.4Z,1,REGULAR,rebound,defensive,[],1628404,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0022100990,24.0
4,9,PT11M25.00S,2022-03-10T03:11:37.5Z,1,REGULAR,2pt,Jump Shot,[],1630314,8.886334,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0022100990,35.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537,651,PT00M19.50S,2022-03-10T05:23:26.8Z,4,REGULAR,stoppage,out-of-bounds,[],0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0022100990,2860.5
538,652,PT00M16.70S,2022-03-10T05:23:34.9Z,4,REGULAR,3pt,Jump Shot,['2ndchance'],1630553,88.255585,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0022100990,2863.3
539,653,PT00M14.00S,2022-03-10T05:23:37.6Z,4,REGULAR,rebound,defensive,[],1627823,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0022100990,2866.0
540,654,PT00M00.00S,2022-03-10T05:23:52.9Z,4,REGULAR,period,end,[],0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0022100990,2880.0


In [62]:
rotation

,GAME_ID,TEAM_ID,TEAM_CITY,TEAM_NAME,PERSON_ID,PLAYER_FIRST,PLAYER_LAST,IN_TIME_REAL,OUT_TIME_REAL,PLAYER_PTS,PT_DIFF,USG_PCT
0,0022100990,1610612762,Utah,Jazz,200752,Rudy,Gay,513.0,947.0,2,0.0,0.214
1,0022100990,1610612762,Utah,Jazz,200752,Rudy,Gay,1874.0,2468.0,7,-4.0,0.292
2,0022100990,1610612762,Utah,Jazz,201144,Mike,Conley,0.0,288.0,0,5.0,0.091
3,0022100990,1610612762,Utah,Jazz,201144,Mike,Conley,513.0,840.0,3,1.0,0.182
4,0022100990,1610612762,Utah,Jazz,201144,Mike,Conley,1200.0,1812.0,2,19.0,0.143
5,0022100990,1610612762,Utah,Jazz,202355,Hassan,Whiteside,288.0,513.0,2,8.0,0.111
6,0022100990,1610612762,Utah,Jazz,202355,Hassan,Whiteside,947.0,1200.0,2,9.0,0.125
7,0022100990,1610612762,Utah,Jazz,202355,Hassan,Whiteside,1812.0,2305.0,2,2.0,0.105
8,0022100990,1610612762,Utah,Jazz,202711,Bojan,Bogdanovic,0.0,513.0,7,13.0,0.316
9,0022100990,1610612762,Utah,Jazz,202711,Bojan,Bogdanovic,947.0,1874.0,20,30.0,0.355


In [102]:
test_pbp_w_player = pd.DataFrame()
test_pbp_wo_player = pd.DataFrame()

player_rotation = rotation[rotation['PERSON_ID'] == player_id]

for row in player_rotation.iterrows():
    row = row[1]
    with_player = test_pbp[
        test_pbp['time'].between(
            row['IN_TIME_REAL'], row['OUT_TIME_REAL'], inclusive='both'
        )
    ]

    test_pbp_w_player = pd.concat([test_pbp_w_player, with_player])


test_pbp_wo_player = test_pbp[
    ~(test_pbp['actionNumber'].isin(test_pbp_w_player['actionNumber']))
]

In [103]:
player_rotation

,GAME_ID,TEAM_ID,TEAM_CITY,TEAM_NAME,PERSON_ID,PLAYER_FIRST,PLAYER_LAST,IN_TIME_REAL,OUT_TIME_REAL,PLAYER_PTS,PT_DIFF,USG_PCT
10,0022100990,1610612762,Utah,Jazz,203497,Rudy,Gobert,0.0,288.0,0,5.0,0.000
11,0022100990,1610612762,Utah,Jazz,203497,Rudy,Gobert,513.0,947.0,5,0.0,0.286
12,0022100990,1610612762,Utah,Jazz,203497,Rudy,Gobert,1200.0,1812.0,4,19.0,0.143


In [104]:
print(len(test_pbp_w_player))

282


In [105]:
test_pbp_w_player

,actionNumber,clock,timeActual,period,periodType,actionType,subType,qualifiers,personId,x,...,officialId,foulPersonalTotal,foulTechnicalTotal,foulDrawnPlayerName,foulDrawnPersonId,blockPlayerName,blockPersonId,value,game_id,time
0,2,PT12M00.00S,2022-03-10T03:11:03.6Z,1,REGULAR,period,start,[],0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0022100990,0.0
1,4,PT11M58.00S,2022-03-10T03:11:05.4Z,1,REGULAR,jumpball,recovered,[],1626220,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0022100990,2.0
2,7,PT11M38.00S,2022-03-10T03:11:24.4Z,1,REGULAR,3pt,Jump Shot,[],1628378,71.698423,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0022100990,22.0
3,8,PT11M36.00S,2022-03-10T03:11:26.4Z,1,REGULAR,rebound,defensive,[],1628404,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0022100990,24.0
4,9,PT11M25.00S,2022-03-10T03:11:37.5Z,1,REGULAR,2pt,Jump Shot,[],1630314,8.886334,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0022100990,35.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343,411,PT05M48.00S,2022-03-10T04:35:56.5Z,3,REGULAR,substitution,out,[],1630190,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0022100990,1812.0
344,412,PT05M48.00S,2022-03-10T04:35:56.5Z,3,REGULAR,substitution,in,[],1630235,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0022100990,1812.0
345,413,PT05M48.00S,2022-03-10T04:35:56.5Z,3,REGULAR,substitution,in,[],202355,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0022100990,1812.0
346,414,PT05M48.00S,2022-03-10T04:35:56.5Z,3,REGULAR,substitution,in,[],1630570,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0022100990,1812.0


In [109]:
test_pbp.shape, test_pbp_w_player.shape, test_pbp_wo_player.shape

((542, 56), (282, 56), (260, 56))

In [106]:
test_pbp_wo_player[test_pbp_wo_player['personId'] == player_id]

,actionNumber,clock,timeActual,period,periodType,actionType,subType,qualifiers,personId,x,...,officialId,foulPersonalTotal,foulTechnicalTotal,foulDrawnPlayerName,foulDrawnPersonId,blockPlayerName,blockPersonId,value,game_id,time


In [107]:
test_pbp_wo_player[(test_pbp_wo_player['actionType'] == 'substitution') & (test_pbp_wo_player['subType'] ==  'out') &
         (test_pbp_wo_player['personId'] == player_id)]

,actionNumber,clock,timeActual,period,periodType,actionType,subType,qualifiers,personId,x,...,officialId,foulPersonalTotal,foulTechnicalTotal,foulDrawnPlayerName,foulDrawnPersonId,blockPlayerName,blockPersonId,value,game_id,time


In [86]:
test_pbp_wo_player

,actionNumber,clock,timeActual,period,periodType,actionType,subType,qualifiers,personId,x,...,officialId,foulPersonalTotal,foulTechnicalTotal,foulDrawnPlayerName,foulDrawnPersonId,blockPlayerName,blockPersonId,value,game_id,time
61,71,PT06M59.00S,2022-03-10T03:21:14.7Z,1,REGULAR,2pt,Layup,"['pointsinthepaint', 'fromturnover']",1628378,91.409330,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0022100990,301.0
62,73,PT06M47.00S,2022-03-10T03:21:29.7Z,1,REGULAR,foul,personal,[],202355,NaN,...,1626300.0,1.0,0.0,Watford,1630570.0,NaN,NaN,NaN,0022100990,313.0
63,75,PT06M41.00S,2022-03-10T03:21:49.8Z,1,REGULAR,2pt,Jump Shot,[],1630314,17.559133,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0022100990,319.0
64,76,PT06M20.00S,2022-03-10T03:22:10.7Z,1,REGULAR,3pt,Jump Shot,[],202711,92.723390,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0022100990,340.0
65,78,PT06M12.00S,2022-03-10T03:22:22.5Z,1,REGULAR,foul,personal,[],1628378,NaN,...,1627962.0,1.0,0.0,Hart,1628404.0,NaN,NaN,NaN,0022100990,348.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
537,651,PT00M19.50S,2022-03-10T05:23:26.8Z,4,REGULAR,stoppage,out-of-bounds,[],0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0022100990,2860.5
538,652,PT00M16.70S,2022-03-10T05:23:34.9Z,4,REGULAR,3pt,Jump Shot,['2ndchance'],1630553,88.255585,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0022100990,2863.3
539,653,PT00M14.00S,2022-03-10T05:23:37.6Z,4,REGULAR,rebound,defensive,[],1627823,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0022100990,2866.0
540,654,PT00M00.00S,2022-03-10T05:23:52.9Z,4,REGULAR,period,end,[],0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0022100990,2880.0
